In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
from keras.datasets import mnist
# seed for reproducing same results
seed = 785
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

In [3]:
import os 
os.getcwd()


'C:\\Users\\HP\\OCR\\ocr-project'

In [4]:
dataset = np.loadtxt('../mnist_train.csv', delimiter=',')
#X = dataset[:,0:784]
#Y = dataset[:,0]

In [5]:
X = np.array(dataset[:,1:])
Y = np.reshape(dataset[:,0],(len(dataset[:,0]),1))

In [16]:
Y[20:30]

array([[4.],
       [0.],
       [9.],
       [1.],
       [1.],
       [2.],
       [4.],
       [3.],
       [2.],
       [7.]])

In [9]:
import cv2

In [19]:
xx1 = np.reshape(X[27,:],(28,28))

In [20]:
cv2.imshow('xyz',xx1)
cv2.waitKey()

-1

In [25]:
np.unique(Y[:,0])

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [27]:
#import pandas as pd

In [29]:
#dataset1=pd.read_csv('../train.csv')
#print(dataset1.head())

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         

In [31]:
#dataset1.shape

(42000, 785)

In [39]:
#randomly shuffling dataset
permutations = list(np.random.permutation(X.shape[0]))
X_shuffled = X[permutations,:]
Y_shuffled = Y[permutations,:]

In [40]:
x_train,x_test,y_train,y_test=train_test_split(X_shuffled,Y_shuffled,test_size=0.20)

In [59]:
train_feature = x_train
test_feature = x_test
train_label = y_train
test_label = y_test

In [60]:
num_classes = 10
train_feature = train_feature.reshape(train_feature.shape[0], 28, 28, 1)
test_feature = test_feature.reshape(test_feature.shape[0], 28, 28, 1)

train_feature = train_feature / 255
test_feature = test_feature / 255
# one hot encode outputs
train_label = np_utils.to_categorical(train_label,num_classes)
test_label = np_utils.to_categorical(test_label,num_classes)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [61]:
print(train_feature.shape)

(48000, 28, 28, 1)


In [9]:
#from keras import backend as K
#K.set_image_dim_ordering('tf')

In [66]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

In [67]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
__________

In [68]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(train_feature,train_label,validation_data=(test_feature,test_label), epochs=5, batch_size=200, verbose=1)


In [69]:
#data augmentation... randomly distorting data to create more data
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()

In [70]:
train_generator = gen.flow(train_feature,train_label, batch_size=64)
test_generator = test_gen.flow(test_feature, test_label, batch_size=64)

In [71]:
model.fit_generator(train_generator, steps_per_epoch=60000//64, epochs=5, 
                    validation_data=test_generator, validation_steps=10000//64)

Epoch 1/5
937/937 [==============================] - 660s 704ms/step - loss: 0.1231 - acc: 0.9623 - val_loss: 0.0734 - val_acc: 0.9764
Epoch 2/5
937/937 [==============================] - 719s 768ms/step - loss: 0.0516 - acc: 0.9837 - val_loss: 0.0308 - val_acc: 0.9911
Epoch 3/5
937/937 [==============================] - 730s 779ms/step - loss: 0.0421 - acc: 0.9866 - val_loss: 0.0336 - val_acc: 0.9896
Epoch 4/5
937/937 [==============================] - 722s 770ms/step - loss: 0.0373 - acc: 0.9883 - val_loss: 0.0267 - val_acc: 0.9922
Epoch 5/5
937/937 [==============================] - 831s 887ms/step - loss: 0.0315 - acc: 0.9905 - val_loss: 0.0282 - val_acc: 0.9916


In [72]:
score = model.evaluate(test_feature,test_label)
print()
print('Test accuracy: ', score[1])

12000/12000 [==============================] - 48s 4ms/step

Test accuracy:  0.9911666666666666


In [73]:
model.save('numbers2.model')

In [74]:
scores = model.evaluate(test_feature,test_label, verbose=1)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

12000/12000 [==============================] - 44s 4ms/step
CNN Error: 0.88%
